In [57]:
# default_exp fontlearner
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Font Learner

> Diffvg-based learner for font optimisation

In [111]:
#export
from aifont.core import *
from enum import Enum
from fastai.data.all import *
from fastai.vision.all import *
import ffmpeg
from nbdev.showdoc import *
import PIL
import pydiffvg
from pydiffvg import Polygon, Rect, RenderFunction, ShapeGroup
from typing import Callable, List, Protocol, Tuple, Union

## Constants and Utilities

In [84]:
#export

COLOR_BLACK  = tensor(0., 0., 0., 1.)
COLOR_WHITE  = tensor(1., 1., 1., 1.)
EMPTY_TENSOR = tensor([])
RGBA_TO_GS   = tensor(0.2989, 0.5870, 0.1140) # Crude NTSC weights for RGB channels
tensor(0.)  = tensor(0.)

def get_vocab(dls_or_learn: Union[DataLoaders, Learner]) -> List[str]:
    """Utility for getting the vocab from a Learner or DataLoaders."""
    if isinstance(dls_or_learn, Learner): dls_or_learn = dls_or_learn.dls
    vocab = dls_or_learn.vocab
    if type(vocab) == L and type(vocab[0]) == list: 
        if vocab[0] != vocab[1]: warn("The two vocabs in dls do not match! Using the first one.")
        vocab = vocab[0]
    return vocab

class DebugCB(Callback):
    """A `Callback` for debugging a `VectorRenderLayer`."""
    vals = []
    grads = []
    def before_fit(self):
        # self.model[0].params.retain_grad()
        pass
    def before_step(self):
        m = self.model
        self.vals.append(m.params.clone())
        self.grads.append(m.params.grad.clone())
    def plot(self) -> None:
        # val = self.model[0].params[0].item()
        # grad_df.iloc[(grad_df.Vals - val).abs().argmin()]
        num = self.vals[0].size(0)
        def _items(tensor_list, idx): return [x[idx].item() for x in tensor_list]
        plt.figure(figsize=(10,10))
        for i in range(num): plt.scatter(_items(self.vals, i), _items(self.grads, i), label=f"Param {i}")
        plt.legend()

## Rendering

In [60]:
#export
class Scene:
    """Just a utility to hold the different scene arguments together."""
    last_seed: int = None
    def __init__(self, shapes: list[any], shape_groups: list[ShapeGroup], 
        canvas_width = 256, canvas_height = 256, samples = 2):
        assert shapes is not None and len(shapes) != 0
        assert shape_groups is not None and len(shape_groups) != 0
        store_attr()

    def get_scene_args(self) -> list:
        """Get the serialize scene for passing to `pydiffvg.RenderFunction`."""
        return RenderFunction.serialize_scene(self.canvas_width, 
                                              self.canvas_height, 
                                              self.shapes, 
                                              self.shape_groups)

    def render(self, seed: int = None, do_render_grad = False) -> Tensor:
        """Render the scene using pydffiv `RenderFunction` or its
           gradient if `do_render_grad`."""
        if seed is None: seed = random.randint(0, 1e6)
        self.last_seed = seed
        scene_args = self.get_scene_args()
        w = self.canvas_width
        h = self.canvas_height
        s = self.samples
        args = [w, h, s, s, seed, None] + scene_args
        rf = RenderFunction
        return rf.render_grad(torch.ones(w, h, 4, device=pydiffvg.get_device()), *args) \
               if do_render_grad else rf.apply(*args)

    def render_grad(self, seed: int = None) -> Tensor:
        """Render the gradient as raster."""
        return self.render(seed=self.last_seed, do_render_grad=True)
    
add_docs(Scene)

In [61]:
#export
class ImageSaver:
    """Create a callback to pass to `VectorRenderLayer` as `rendered_callback`
       to save rendered images and optionally grads."""
    canvas_height: int = None
    canvas_width: int = None
    iter_files: List[str] = []
    grad_files: List[str] = []
    def __init__(self, folder: str, save_grad = False, iter_name = "iter", 
                 grad_name = "grad"):
        assert folder is not None
        if folder.endswith("/"):
            folder = folder[:-1]
        store_attr()

    def __call__(self, raster: Tensor, batch_i: int, item_i: int, scene: Scene,
                 normalize = False, gamma = 2.2) -> None:
        suffix = f"_{batch_i}_{item_i}"
        self.iter_files.append(self.save_image(raster, f"{self.iter_name}{suffix}", normalize=normalize, gamma=gamma))
        if self.save_grad: 
            grad = scene.render_grad()
            self.grad_files.append(self.save_image(grad, f"{self.grad_name}{suffix}"))

    def save_image(self, raster: Tensor, filename: str, normalize = False, gamma = 2.2) -> Str:
        """Save the `raster` tensor as image file and return filename used."""
        if not self.canvas_height:
            self.canvas_height = raster.size(0)
            self.canvas_width  = raster.size(1)
        fn = f"{self.folder}/{filename}.png"
        pydiffvg.imwrite(raster.cpu(), fn, normalize=normalize, gamma=gamma)
        return fn
    
    def render_result_video(self, delete_imgs = False, frame_rate=24, grad=False) -> None:
        """Render intermediate images as a video."""
        files = self.grad_files if grad else self.iter_files
        out = os.path.join(self.folder, "grads.mp4" if grad else "iters.mp4")
        frames = ffmpeg.input('pipe:', r=str(frame_rate))
        process = ffmpeg.input(f"color=c=white:s={self.canvas_width}x{self.canvas_height}", f="lavfi") \
                        .overlay(frames, eof_action="endall") \
                        .output(out) \
                        .overwrite_output() \
                        .run_async(pipe_stdin=True, quiet=True)
        for in_file in files:
            with open(in_file, 'rb') as f: process.stdin.write(f.read())
        # Close stdin pipe - FFmpeg fininsh encoding the output file.
        process.stdin.close()
        process.wait()
        if delete_imgs: 
            for f in files: os.remove(f)
        print("Rendering video done!")

add_docs(ImageSaver)

## DataLoader

Custom DataLoader for getting letter classes.

In [62]:
#export
class LetterDL(DataLoader):
    """A dummy data loader for use with font vector optimisation.
       Pass the same `vocab` as in the OCR model. Batch size defaults
       to `epoch_len`."""
    current_i = 0
    def __init__(self, vocab: CategoryMap, letters: Tuple[str, ...] = ("A",), 
                 epoch_len = 10, bs = 1, **kwargs):
        assert vocab is not None
        super(LetterDL, self).__init__(bs=bs, n = epoch_len * bs, **kwargs)
        self.categorizer = Categorize(vocab=vocab)
        store_attr()

    def create_item(self, s) -> Tuple[TensorCategory, TensorCategory]:
        """Return the CategoryTensor for a random letter from `letters`."""
        if self.current_i == self.n:
            self.current_i = 0
            stop()
        self.current_i += 1
        r = self.categorizer.encodes(random.choice(self.letters))
        # Return x and y as a copy of x
        return r, r.clone()

add_docs(LetterDL)

## Vector Model

The vector model consists of a `FontParamLayer`, which holds the parameters to optimise, and a subclass of `VectorRenderLayer`, which handles the creation of the letter vectors.

> Note that the utility of this bisection is tentative, and the params might as well be contained within the `VectorRenderLayer`.

### Font Parameters

Font parameters can be simple or specified to interpolate to certain ranges. 

The the order they are stored in the parameter layer. For each, a ParamRange tuple containing the min, max and optional mean values is passed, which are used to interpolate the value from the raw value, usually (–1, 1), passed by the parameter layer. Most values are treated as fractions, usually of cap height and, for Height, of canvas height. For standard ranges use the PRange enum.

In [63]:
#export

class ParamInterpolator(GetAttr):
    """Create an interpolator for sigmoid-activated param values passed
       by the `FontParamLayer`. Use `asymmetric` for values whose
       distribution is highest at `min` and tapers towards `max`."""

    noop = False

    def __init__(self, min=None, max=None, mean=None, asymmetric=False):
        if min is None: min = 0.
        if max is None: max = 1.
        if min == 0 and max == 0 and not asymmetric: self.noop = True
        min, max = tensor(min), tensor(max)
        self.v_range = max - min
        store_attr()

    def __call__(self, value: Tensor) -> Tensor:
        return self.interpolate(value)

    def interpolate(self, value: Tensor) -> Tensor:
        """Interpolate `value`."""
        if self.noop: return value
        if self.asymmetric: value = 2 * torch.maximum(value - .5, tensor(0.))
        return self.min + self.v_range * value

# Parameter ranges
PRANGE_DEFAULT = ParamInterpolator( 0.0,   1.0)
PRANGE_SMALL   = ParamInterpolator( 0.0,   0.25)
PRANGE_HALF    = ParamInterpolator( 0.0,   0.5)
PRANGE_BIDIR   = ParamInterpolator(-1.0,   1.0)
PRANGE_BIDIR_H = ParamInterpolator(-0.5,   0.5)
PRANGE_BIDIR_S = ParamInterpolator(-0.25,  0.25)
PRANGE_NONZERO = ParamInterpolator( 0.05,  1.0)
PRANGE_ASYM    = ParamInterpolator( 0.0,   1.0,  asymmetric=True)
PRANGE_ASYM_S  = ParamInterpolator( 0.0,   0.25, asymmetric=True)
PRANGE_STROKE  = ParamInterpolator( 0.01,  0.25)

### Vector Rendering Layer Base

In [86]:
#export

class Normaliser(Protocol):
    """For normalising rasters for the OCR."""
    mean: float
    std: float

class VectorRenderLayerBase(Module):
    """Base for vector render layers. Get's input from a FontParamLayer
       and returns the diffvg rendering. Override `create_scenes` in
       subclasses and save the results in `self.scenes` of which there
       should be `bs`. `forward` calls `render` which renders the
       scenes and permutes to match the OCR model. Note that the workflow
       is based on greyscale images and we're only using the alpha value
       of the diffvg render output.

       Params can be defined simply with `n_params` or an `OrderedDict` 
       `param_specs` that contains each parameter's name and a 
       `ParamInterpolator` to which the activated param values is passed.
       All params have sigmoid activation and, thus, have an output range
       of [0, 1] before possible interpolation.
       
       Init parameters:
       `canvas_width`, `canvas_height`: rastered canvas dims
       `raster_norm`: use the normaliser from the OCR `dls`
       `clip_raster`: whether to clip color values to [0., 1.] as is done when
            saving image (note that the values produced by the render
            function value wildly up to more than 10. so setting this to
            False is advised against)
       `apply_gamma`: whether to apply `gamma` to the color values similarly
            to clipping above
       `n_params`: number of params (with sigmoid activation). Either this or
            `param_specs` must be passed.
       `param_specs`: `OrderedDict` that contains each parameter's name and a 
            `ParamInterpolator` to which the activated param values is passed.
       `vocab`: list of letter strings to which the inputs are matched
       `init_range`: defines the range of the   pre-activation parameter 
            value space when initialised at random by `reset_parameters` 
            centered around the middle.
       `eps`: amount of random jitter added to params, use with care!
       `n_colors_out`: color channels out
       `max_distance`: the maximum fraction [0., 1.] of canvas dims 
            `distance_params` can span
       `fixed_seed`: fixed seed value to pass to `pydiffvg.RenderFunction`
       `gamma`: set to override default gamma of 2.2 for colour images and 1.
            for grayscale ones
       `stroke_width`: stroke width for shape generator helpers
            (note that this is defined as a fraction of `canvas_size`);
            either a float or a tuple of min and max width and used by
            `expand_stroke_width`
       `stroke_color`: default stroke color for shape generator helpers
       `rendered_callback`: set to an ImageSaver to save interim renders"""
    batch_i = -1
    bs: int = None
    debug = False
    i: int # Current item index
    params_with_eps: Tensor = None
    scenes: List[Scene] = []
    sigmoid = torch.nn.Sigmoid()
    stored = []
    x: Tensor = None
    def __init__(self, canvas_width: int, canvas_height: int, vocab: CategoryMap, 
                 param_specs: OrderedDict = None, n_params: int = None, seed: int = None, 
                 init_range = 2., eps: float = None, raster_norm: Normaliser = None, clip_raster = True, 
                 apply_gamma = True, n_colors_out = 1, max_distance = 1., fixed_seed: int = None, 
                 gamma: float = None, stroke_width: Union[float, Tuple[float, float]] = 1./28, 
                 stroke_color = COLOR_BLACK, 
                 rendered_callback: Callable[[Tensor, int, int, Scene, bool, float], None] = None):
        if max_distance is None: max_distance = 1.
        assert max_distance <= 1.
        if param_specs is not None: n_params = len(param_specs)
        assert n_params is not None and vocab is not None
        if seed is not None: torch.random.manual_seed(seed)
        super(VectorRenderLayerBase, self).__init__()
        self.param_names = list(param_specs.keys())
        self.weight = torch.nn.Parameter(torch.empty(n_params))
        self.canvas_size = max(canvas_width, canvas_height)
        stroke_width = tensor(stroke_width)
        if canvas_width != canvas_height: 
            warn(f"When canvas is not square ({canvas_width}x{canvas_height}), "
                  "some dimensions may be expanded outside it.")
        if gamma is None: gamma = 1. if n_colors_out == 1 else 2.2
        store_attr()
        self.reset_parameters()

    def __repr__(self):
        if self.param_specs is None:
            p_strings = [f"  {x.item()} ({self.sigmoid(x).item()})" for x in self.weight]
        else:
            p_strings = [f"  {i} {k}: {self.weight[i].item()} ({self.get_param_by_name(k, no_eps=True).item()})" \
                         for i,k in enumerate(self.param_specs.keys())]
        return "\n".join([f"{self.__class__.__name__} with params:"] + p_strings)

    @property
    def params(self) -> Tensor:
        """A synomym for `weight`."""
        return self.weight

    def reset_parameters(self):
        """Randomly init the parameters around the middle of possible values."""
        self.weight.data.uniform_(-self.init_range, self.init_range)

    def get_item(self, i: int) -> Tensor:
        """Get item `i` in the batch `x`"""
        return self.x[i]

    def get_letter(self, i: int = None) -> string:
        """Get letter as string for item `i`."""
        if i is None: i = self.i
        return self.vocab[self.get_input(i).int().item()]

    def get_param_by_name(self, name: str, i: int = None, no_eps = False) -> Tensor:
        """Get interpolated param value by `name` for item `i`."""
        assert self.param_specs is not None and name in self.param_specs
        idx = self.get_param_index(name)
        v = self.get_params(i, no_eps=no_eps)[idx]
        return self.param_specs[name](v)

    def set_param(self, name: str, value: Union[float, Tensor]) -> None:
        """Set the value for the named param. Mostly for debugging."""
        assert self.param_specs is not None and name in self.param_specs
        idx = self.get_param_index(name)
        self.params.data[idx] = tensor(value)

    def get_param_index(self, name: str) -> int:
        """Get the index of the named param."""
        return self.param_names.index(name)

    def get_params(self, i: int, no_eps = False) -> Tensor:
        """Get params for item `i` in the batch passed by 
           `FontParamLayer` as part of `x`"""
        assert not no_eps or i is None, "When using no_eps, don't use i"
        if i is None and not no_eps: i = self.i
        return self.sigmoid(self.params if no_eps or not self.eps else self.params_with_eps[i])
        
    def get_input(self, i: int) -> Tensor:
        """Get the letter category for item `i` in `x`"""
        return self.x[i]

    def add_eps(self) -> None:
        """Apply random eps to params, which differs for each item in the batch. 
           Cf. diffvg/apps/generative modeling/rendering.render_lines"""
        assert self.bs is not None
        if not self.eps: return # params_with_eps defaults to params
        sz = (self.bs,) + self.params.size()
        self.params_with_eps = self.params.expand(sz) + self.eps * torch.randn(sz)

    def expand_distance(self, vals: Tensor) -> Tensor:
        """Expand values to a central `self.max_distance` fraction of the canvas.
           Coordinates originate from NW."""
        return self.canvas_size * vals if self.max_distance == 1. else \
               self.canvas_size * ((1 - self.max_distance) / 2 + vals * self.max_distance)

    def expand_stroke_width(self, vals: Tensor = None) -> Tensor:
        """Expand `vals`,  based on `[min_stroke_width, max_stroke_width] * canvas_height`.
           Note that we divide by two so that result is in line with traditional usage in
           vector software."""
        if vals is None: 
            warn("Using default stroke width in rendering.")
            vals = tensor(1.)
        w = vals * self.stroke_width if self.stroke_width.ndim == 0 \
            else self.stroke_width[0] + vals * (self.stroke_width[1] - self.stroke_width[0])
        return w * self.canvas_size / 2

    def normalise_raster(self, raster: Tensor) -> Tensor:
        """Apply normalisation to `raster`. Not useful for grayscale letters."""
        if not self.raster_norm: return raster
        return (raster - self.raster_norm.mean) / self.raster_norm.std 

    def forward(self, x) -> Tensor:
        """Render letters defined in `x`."""
        # Convert the input x to size (bs, 1) if it's one-dimensional
        if x.ndim == 1: x = x.unsqueeze(1)
        elif x.ndim != 2: raise ValueError("Input can only be 1- or 2-dimensional.")
        self.batch_i += 1
        self.x = x
        self.bs = x.size(0)
        self.add_eps()
        self.scenes = [None] * self.bs
        self.create_scenes()
        return self.render()

    def create_scenes(self) -> None:
        """Override this in subclasses to create the vector scenes for the letters."""
        raise NotImplementedError()

    def create_line_scene(self, *shapes) -> Scene:
        """Create a simple line-drawing Scene with shapes."""
        shape_groups = [self.create_line_group(*shapes)]
        return self.create_scene_from_groups(shapes, shape_groups)

    def create_line_group(self, *shapes, stroke_color=None, id_offset=0) -> ShapeGroup:
        """Create a ShapeGroup from shapes for line drawing."""
        assert len(shapes) and type(shapes[0]) not in (tuple, list), "Unpack shapes"
        if stroke_color is None: stroke_color = self.stroke_color
        return ShapeGroup(shape_ids=tensor([x + id_offset for x in range(len(shapes))]),
                          fill_color=None,
                          stroke_color=stroke_color,
                          use_even_odd_rule=False)

    def create_fill_group(self, *shapes, fill_color=None, id_offset=0) -> ShapeGroup:
        """Create a ShapeGroup from shapes for filling."""
        assert len(shapes) and type(shapes[0]) not in (tuple, list), "Unpack shapes"
        if fill_color is None: fill_color = self.stroke_color
        return ShapeGroup(shape_ids=tensor([x + id_offset for x in range(len(shapes))]),
                          fill_color=fill_color,
                          stroke_color=None,
                          use_even_odd_rule=True)

    def create_scene_from_groups(self, shapes, shape_groups) -> list:
        """Create a scene from `shapes` and `shape_groups`."""
        # Check that there are no duplicate ids
        all_ids = torch.concat([x.shape_ids for x in shape_groups])
        assert all_ids.numel() == all_ids.unique().numel()
        return Scene(shapes=shapes, shape_groups=shape_groups, 
                     canvas_width=self.canvas_width, canvas_height=self.canvas_height)

    def create_mixed_scene(self, line_shapes=[], fill_shapes=[], stroke_color=None, fill_color=None, bg_color=None) -> list:
        """Create a scene that has both `line_shapes` and `fill_shapes` and optionally a background."""
        shapes = line_shapes + fill_shapes
        shape_groups = []
        id_offset = 0
        if bg_color is not None:
            bg = Rect(tensor(0., 0.), tensor(self.canvas_width, self.canvas_height), stroke_width=tensor(0.))
            shapes.insert(0, bg)
            shape_groups.append(self.create_fill_group(bg, fill_color=bg_color, id_offset=id_offset))
            id_offset += 1
        if len(line_shapes) > 0: 
            shape_groups.append(self.create_line_group(*line_shapes, stroke_color=stroke_color, id_offset=id_offset))
            id_offset += len(line_shapes)
        if len(fill_shapes) > 0:
            shape_groups.append(self.create_fill_group(*fill_shapes, fill_color=fill_color, id_offset=id_offset))
        return self.create_scene_from_groups(shapes, shape_groups)

    def create_line_scene_from_points(self, *point_tensors, stroke_width=None, is_closed=False, expand_distance=False) -> Scene:
        """Shorthand for `create_line_scene` by passing `point_tensors` that
           are converted to polygons."""
        return self.create_line_scene(*self.points_to_polygons(*point_tensors, stroke_width=stroke_width, 
                                                               is_closed=is_closed, expand_distance=expand_distance))

    def points_to_polygons(self, *point_tensors, stroke_width=None, is_closed=False, expand_distance=False) -> List[Polygon]:
        """Convert `point_tensors` to a List of pydiffvg Polygons."""
        return [Polygon(points=self.expand_distance(pt) if expand_distance else pt,
                        stroke_width=self.expand_stroke_width() if stroke_width is None else stroke_width,
                        is_closed=is_closed) \
                for pt in point_tensors]

    def render(self) -> Tensor:
        """Render `self.scenes` as a raster tensor using pydiffvg."""
        assert self.scenes is not None and len(self.scenes) == self.bs
        cols = self.n_colors_out
        output = torch.zeros(self.bs, cols, self.canvas_width, self.canvas_height) # .requires_grad_()
        for i, s in enumerate(self.scenes):
            raster = s.render(seed=self.fixed_seed)
            if self.rendered_callback: 
                self.rendered_callback(raster=raster, batch_i=self.batch_i, item_i=i, scene=s, normalize=False, gamma=self.gamma)
            if cols in (1, 3):
                # Output is w,h,rgba, where with values in 0.-1. (and black thus 0., 0., 0., 1.)
                # First, we apply alpha by mixing output with white in that proportion
                if self.debug: self.stored.append(raster.clone())
                alpha = raster[:, :, 3].unsqueeze(2).expand(-1, -1, 4)
                white = torch.full_like(raster, 1.) * (1. - alpha)
                raster = (white + raster * alpha)[:, :, :3] # Now raster is w,h,rgb
                if cols == 1: # Convert to grayscale if needed
                    raster *= RGBA_TO_GS # This is a crude NTSC sampling to grayscale
                    raster = raster.sum(-1, keepdims=True)
                raster = raster.permute(2, 0, 1) # Order channel-first
            elif cols != 4: raise NotImplementedError(f"n_colors_out '{cols}' can only be 1, 3 or 4.")
            raster = self.normalise_raster(raster)
            if self.clip_raster: raster = raster.clip(0., 1.)
            if self.apply_gamma: 
                if cols == 1: raster = raster.pow(1.0/self.gamma)
                else: raster[:,:,:3] = raster[:,:,:3].pow(1.0/self.gamma)
            # assert raster.requires_grad
            output[i] = raster
        return output

add_docs(VectorRenderLayerBase)

## Loss Function

In [65]:
#export
class OCRLoss(CrossEntropyLossFlat):
    """Softmaxed CrossEntropyLossFlat between `ocr_model`'s prediction
       and target category. Use after `VectorRenderLayerBase`."""
    stored: List[Tuple[float, int, float, Tensor]] = []
    def __init__(self, ocr_model, debug = False, **kwargs):
        super(OCRLoss, self).__init__(**kwargs)
        assert ocr_model is not None
        ocr_model.eval()
        store_attr("ocr_model, debug")

    def __call__(self, inp, target):
        pred = self.activation(self.ocr_model(inp))
        loss = super(OCRLoss, self).__call__(pred, target)
        if self.debug: self.stored.append((loss.item(), pred[0].argmax().item(), pred[0].max().item(), pred[0].detach()))
        return loss

def param_loss(x: Tensor, loss_start=4., loss_factor=1.) -> Tensor:
    """Calculate a linear loss for abs values above `loss_start` multiplied
       by `loss_factor`."""
    return loss_factor * torch.maximum(x.abs() - loss_start, tensor(0.)).sum()

class ParamLoss(Module):
    """Calculate a loss based on extreme parameter values."""
    def __init__(self, vector_model: VectorRenderLayerBase, loss_start=4., loss_factor=1., **kwargs):
        super(ParamLoss, self).__init__(**kwargs)
        assert vector_model is not None
        store_attr("vector_model,loss_start,loss_factor")

    def forward(self, *args):
        return param_loss(self.vector_model.params, self.loss_start, self.loss_factor)
        
class OCRAndParamLoss(Module):
    """Combined OCR and param loss."""
    def __init__(self, ocr_model, vector_model: VectorRenderLayerBase, loss_start=4., loss_factor=1., debug = False, **kwargs):
        super(OCRAndParamLoss, self).__init__(**kwargs)
        self.ocr_loss = OCRLoss(ocr_model=ocr_model, debug=debug, **kwargs)
        self.param_loss = ParamLoss(vector_model=vector_model, loss_start=loss_start, loss_factor=loss_factor, **kwargs)

    def forward(self, inp, target):
        return self.ocr_loss(inp, target) + self.param_loss(inp, target)

## Learner

In [66]:
#export
class VectorLearner(Learner):
    """A simple extension to Learner offering some utility methods."""
    def __init__(self, image_saver=None, **kwargs):
        super(VectorLearner, self).__init__(**kwargs)
        store_attr("image_saver")
    
    @property
    def vocab(self) -> List[str]:
        return self.dls.vocab

    def reset_parameters(self):
        """Shortcut for `self.model.reset_parameters`."""
        self.model.reset_parameters()

    def set_param(self, *args, **kwargs):
        """Shortcut for `self.model.set_param`."""
        self.model.set_param(*args, **kwargs)

    def render_letter(self, letter: str = "A", scale: float = None) -> PIL.Image:
        """Render a letter using the current vector model."""
        inp = tensor([self.vocab.index(letter)])
        m = self.model
        trn = m.training
        m.eval()
        with torch.no_grad(): img = m(inp).squeeze().clip(0., 1.) * 255
        m.train(trn)
        pil_img = PILImageBW.create(img).convert('RGB')
        if scale is not None: pil_img = pil_img.resize((round(pil_img.width * scale), round(pil_img.height * scale)), resample=0)
        return pil_img

    def render_result_video(self, **kwargs):
        """Shortcut for `self.image_saver.render_result_video`"""
        assert self.image_saver is not None
        self.image_saver.render_result_video(**kwargs)

    def calculate_losses(self, n = 20, param_ranges: List[Union[Tuple[float, float], float]] = None) -> pd.DataFrame:
        """Output loss statistics and predictions for different param values.
           If `param_ranges` is supplied, it should contain the min and max
           values to use for each parameter or a fixed value."""
        assert n > 1
        assert self.loss_func.debug, "Debug must be enabled for the loss function."
        model = self.model
        model.eval()
        pl = model[0]
        n_pars = pl.n_params
        param_f = torch.full((n_pars,), 4)  if param_ranges is None else tensor([0. if type(x) is float else x[1] - x[0] for x in param_ranges])
        param_c = torch.full((n_pars,), -2) if param_ranges is None else tensor([x  if type(x) is float else x[0] for x in param_ranges])
        x,y = self.dls.one_batch()
        stats = []
        for i in range(n):
            p_vals = param_c + param_f * i / (n - 1)
            pl.params.data = p_vals
            # Vector prediction
            p = model(x)
            _ = self.loss_func(p, y)
            d = {
                "loss": self.loss_func.stored[-1][0],
                "pred": self.vocab[self.loss_func.stored[-1][1]],
                "pred_activation": self.loss_func.stored[-1][2]
                }
            for j in range(n_pars): d[f"param_{j}"] = p_vals[j].item()
            stats.append(d)
        return pd.DataFrame(stats)

add_docs(VectorLearner)

## Export

In [116]:
#hide
from nbdev.export import notebook2script; notebook2script()

Converted 01_fontlearnertests.ipynb.
Converted 02_lettervectors.ipynb.
Converted aifont_core.ipynb.
Converted aifont_fontlearner.ipynb.
Converted aifont_fontsampler.ipynb.
Converted aifont_ocrlearner.ipynb.
Converted index.ipynb.
